# Introduction

mplementing kernels designed in Automatic forecasting using Gaussian Process [1] and test results with the M4 dataset. For now, only small sections of the M4 training and testing datasets are being used.



# Data loading and visualization
The data is available at the following path.

In [ ]:
from pandas.core.groupby.generic import DataFrameGroupBy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
!pip install gpytorch # This is to allow gpytorch to work on any Google Colab Notebook
import gpytorch as gp
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 16.7 MB/s eta 0:00:00


Setting up the full m4 dataset and any subsets (ex. monthly, weekly, hourly, etc.) that might be analyzed.

In [ ]:
random.seed(0)
df = pd.read_csv('/content/M4-info.csv') # This is for the m4 dataset as a whole
df_test = pd.read_csv("/content/Monthly-test.csv") # Monthly dataset for testing purposes
df_train = pd.read_csv("/content/Monthly-train.csv") # Monthly dataset for training purposes

ParserError: ignored

In [ ]:
df # Just to review the M4_Info dataset

In [ ]:
df_train # Good too see how the training set compares to the info set.

In [ ]:
df_test # Good too see how the testing set compares to the info set.

Select series_id for one product, and we'll fit GP as a univariate model. Code not used for now.

In [ ]:
''' Code Not Used as datasets are set up separately. Might used later if applicable
series_id = 'M2573'
num_test = 18
df = df[df['M4id']==series_id] # M4id corresponds to the series_id in the original m3 dataset.
df
'''

## Setting a new dataset with one row from the training dataset
We visualize the data of one row in the M4 dataset and put that rows into its own dataset, which will now be used for the training dataset.


In [ ]:
df_2 = df_train.loc[2574].to_frame() # This is a dataframe housing the row that I chose, which I believe is the one with the most elements.
df_2 = df_2.dropna()
df_2 = pd.DataFrame(df_2) # Just confirming that we made a new dataframe
df_2["index"] = df_2.index # There are the indexes corresponding to each element
items = df_2.iloc[:, 0]
df_2 = df_2.assign(items=items)
df_2 = df_2.drop("V1")
df_2['index'] = df_2['index'].str.replace('V', '') # Removing V from each index so that indecies are numbers
df_2

In [ ]:
plt.rcParams['figure.figsize'] = 12, 8
sns.lineplot(x='index', y='items', data=df_2) # Revised based on how I formatted the dataset.
plt.xlabel('Month - Index of Each Element')
plt.ylabel('Value')

## Plotting the auto-correlation
We plot the auto correlcation function below.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
acf = plot_acf(df_2['items'])

## Setting a new dataset with one row from the testing dataset
We visualize the data of one row in the M4 dataset and put that rows into its own dataset, which will now be used for the testing dataset.


In [ ]:
df_3 = df_test.loc[2574].to_frame() # This is a dataframe housing the row that I chose.
df_3 = df_3.dropna()
df_3 = pd.DataFrame(df_3) # Just confirming that we made a new dataframe
df_3["index"] = df_3.index # There are the indexes corresponding to each element
items = df_3.iloc[:, 0]
df_3 = df_3.assign(items=items)
df_3 = df_3.drop("V1")
df_3['index'] = df_3['index'].str.replace('V', '') # Removing V from each index so that indecies are numbers
j = 1855
for i in range(len(df_3['index'])): # This makes the indecies of the testing dataset to be after the training set's indecies.
  df_3['index'][i] = j
  j = j + 1
df_3 # Change the indecies of the testing dataset for its first index to be after than the last index of training dataset.


In [ ]:
# This is for the testing dataset
plt.rcParams['figure.figsize'] = 12, 8
sns.lineplot(x='index', y='items', data=df_3) # Revised based on how I formatted the dataset.
plt.xlabel('Month - Index of Each Element')
plt.ylabel('Value')

## **Data Testing - Basic**
We prepare the training set and test set data and work with using basic GPs. The data needs to be converted to tensors to work, so some precision of the data is lost. The data is converted to int32 tensors.

In [ ]:
# From the newly created training dataset df_2
x_train = df_2["index"] # Months
x_train = torch.from_numpy(x_train.values.astype(np.float32))

y_train = df_2["items"] # Values
y_train = torch.from_numpy(y_train.values.astype(np.float32))

# From the newly created testing dataset df_3
x_test = df_3["index"] # Months
x_test = torch.from_numpy(x_test.values.astype(np.float32))

y_test = df_3["items"] # Values
y_test = torch.from_numpy(y_test.values.astype(np.float32))

In [ ]:
# Using the tutorial code for the simplified testing and training with the dataset.
# Will need to change the GPModel being used as needed.

# Fit the right side of the data at the change in the training graph. See sharp turn.
class GPModel(gp.models.ExactGP):
  def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gp.means.ConstantMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel() * gp.kernels.PeriodicKernel() + gp.kernels.RQKernel()) # Most effective of the combinations I tried, but by the 100th test, the negative LL is still 3023.869.
        # elf.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel() * gp.kernels.PeriodicKernel() + gp.kernels.LinearKernel() * gp.kernels.PeriodicKernel()) - This gives low initial losses, but they increase once we go past the 50th round.
        # self.covar_module.initialize_from_data(x_train.type(torch.float32), y_train.type(torch.float32)) # Only for the SpectralMixture Kernel
  def forward(self, x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return gp.distributions.MultivariateNormal(mean_x, covar_x)
  def mse(y_true, y_pred):
    # Mean squared error function.
    y_pred_tensor = y_pred.sample()
    return ((y_true - y_pred_tensor)**2).mean()

  def mae(y_true, y_pred):
    # Mean absolute error function.
    y_pred_tensor = y_pred.sample()
    return torch.abs((y_true - y_pred_tensor)**2).mean()

  def crps(y_true, y_pred):
    # Continuous ranked probability score function.
    y_pred_tensor = y_pred.sample()
    return torch.mean(torch.distributions.Normal(y_pred_tensor, torch.sqrt(y_pred_tensor.std())).cdf(y_true))

likelihood = gp.likelihoods.GaussianLikelihood()
model = GPModel(x_train, y_train, likelihood)

In [ ]:
# This takes between 1-16 minutes on my laptop, for reference

# Running the notebook in the tutorial's testing framework
import os
smoke_test = ("CI" in os.environ)
training_iter = 2 if smoke_test else 40100 # else __ of training iterations
# 40189 was maximum before runtime failed. Reached a low of 39.363
# This took my computer around 4-6 hours.

# Finding the optimal parameters for the model
model.train()
likelihood.train()

# Using the ADAM optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1) # Change the learning rate as needed. Lower should be better.

# Finding the marginal log likelihood, or the "loss" for GPs
mll = gp.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
  optimizer.zero_grad()
  output = model(x_train)
  loss = -mll(output, y_train)
  loss.backward()
  print("Iter %d/%d - Loss: %.3f" % (i + 1, training_iter, loss.item()))
  optimizer.step()

In [ ]:
# Get into evaluation mode, or the predictive posterior mode
# Add the training and test metrics. Ex. MSE and MAE and CRPS. Test each individually.
# These tests have been incorporated as functions into the GPModel Class

model.eval()
likelihood.eval()

with torch.no_grad(), gp.settings.fast_pred_var():
    # Making predictions
    observed_pred = likelihood(model(x_test))

    # Initializing the plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Getting the upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plotting training data as black stars
    ax.plot(x_train.numpy(), y_train.numpy(), 'k*')
    # Plotting predictive means as blue line
    ax.plot(x_test.numpy(), observed_pred.mean.numpy(), 'b')
    # Shading the region between the lower and upper confidence bounds
    ax.fill_between(x_test.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # ax.set_ylim([0, 2000])
    ax.set_xlim([1400, 2000])
    # plt.fit()
    ax.legend(['Observed Data', 'Mean', 'Confidence'])

# Evaluation for Basic Testing
We measure mean absolute error (MAE), continuous-ranked probability score (CRPS), and log-likelihood (LL) using normalized values on both of the training set and the test set.

In [ ]:
y_pred = model(x_train)
mse_train = GPModel.mse(y_train, y_pred)
mae_train = GPModel.mae(y_train, y_pred)
crps_train = GPModel.crps(y_train, y_pred)

print("MSE train:", mse_train)
print("MAE train:", mae_train)
print("CRPS train:", crps_train)

In [ ]:
# Importing and installing sts so it can be used later on. This does not work. I need to fix this.
!pip install git+https://github.com/jpchen/autoforecasting.git
from sts.data import get_mvn_stats
from torch.distributions import ComposeTransform, ExpTransform

In [ ]:
train_mean, train_var, train_ci = get_mvn_stats(model.predict(x_train))
test_mean, test_var, test_ci = get_mvn_stats(model.predict(x_test))

Here we compute MAE for both training data and test data.

In [ ]:
from sts.metrics import MAE
MAE_train = MAE(y_train.tensor, train_mean).item()
MAE_test = MAE(y_test.tensor, test_mean).item()
print(f'Mean absolute error for training set is: {MAE_train:.3f}')
print(f'Mean absolute error for test set is: {MAE_test:.3f}')

Here we compute CRPS for both training data and test data.

In [ ]:
from sts.crps import crps_gaussian
crps_train = crps_gaussian(y_train.tensor, train_mean, train_var.sqrt())
crps_test = crps_gaussian(y_test.tensor, test_mean, test_var.sqrt())
print(f'Continuous ranked probability score for training set is: {crps_train.mean().item():.3f}')
print(f'Continuous ranked probability score for test set is: {crps_test.mean().item():.3f}')

Here we compute log-likelihood for both training data and test data.



In [ ]:
from sts.metrics import log_likelihood_error
ll_train = log_likelihood_error(train_mean, y_train.tensor, train_var).item()
ll_test = log_likelihood_error(test_mean, y_test.tensor, test_var).item()
print(f'Log-likelihood for training set is: {ll_train:.3f}')
print(f'Log-likelihood for test set is: {ll_test:.3f}')

In [ ]:
scores = [[MAE_train, crps_train.mean().item(), ll_train], [MAE_test, crps_test.mean().item(), ll_test]]
df_meature = pd.DataFrame(scores, columns = ['MAE', 'CPRS', 'LL'])
df_meature


The provided table displays the metric values, with the first line representing the training set and the second line representing the test set. It's worth noting that lower values are preferable for MAE and CRPS, while higher values are desired for LL.

Upon closer examination, we have observed that the training set exhibits higher accuracy compared to the test set. To address this issue and enhance overall performance, we propose implementing a solution that involves assigning priors to the hyperparameters. Additionally, we will leverage hierarchical priors to capitalize on the joint information present in the univariate time series data.

By incorporating these priors into our model, we aim to optimize its performance and reduce the disparity between the accuracy of the training and test sets. This approach will allow us to better capture the underlying patterns and relationships within the data, ultimately improving the model's predictive capabilities.

## **Data Testing - Less Basic**
We prepare the training set and test set data and work with using non-standard GPs.

In [ ]:
from sts.gp.model import AdditiveSpectralMixtureTimeSeriesExactGPModel # Check the equivalent models if they have been replaced.
from sts.gp.kernel import WhiteNoiseKernel
from gpytorch.kernels import LinearKernel, RBFKernel

likelihood = gp.likelihoods.GaussianLikelihood()
model = AdditiveSpectralMixtureTimeSeriesExactGPModel(df_train, likelihood)
model.cov.add_seasonality(
            time_axis="timestamp", period_length=365.25, fix_period=True
        )
model.cov.add_trend(
    time_axis="timestamp", kernel_cls=LinearKernel, name="LinearTrend"
)
model.cov.add_trend(
    time_axis="timestamp",
    kernel_cls=RBFKernel,
    lengthscale=100,
    fix_lengthscale=True,
    name="RBFTrend",
)
model.cov.add_spectral_mixture(
    time_axis="timestamp",
    num_mixtures=2,
    train_x=df_train.tensor,
    train_y=y_train.tensor,
    name="SM1",
)
model.cov.add_spectral_mixture(
    time_axis="timestamp",
    num_mixtures=2,
    train_x=df_train.tensor,
    train_y=y_train.tensor,
    name="SM2",
)


# Training loop
We train our model with our training set, and show the training time below.

In [ ]:
learning_rate = 0.01
num_epochs = 3000
trainer = model.train_init(torch.optim.Adam(model.trainable_params, lr=learning_rate))

def train():
    for epoch in range(num_epochs):
        loss = trainer(df_train, y_train)
        if (epoch + 1) % 200 == 1:
            print(f'epoch {epoch+1}/{num_epochs}, loss {loss}')

%time train()

# Predictions
We use our model to forecast the test set.



In [ ]:
from sts.data import get_mvn_stats
from torch.distributions import ComposeTransform, ExpTransform

transform = ComposeTransform([y_train.transforms['log_value'].inv, ExpTransform()])
train_mean, train_var, train_ci = get_mvn_stats(model.predict(df_train), transform)
test_mean, test_var, test_ci = get_mvn_stats(model.predict(x_test), transform)

# This will need to be fixed based on how I created the training and testing datsets.
f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(df['timestamp'][:num_trainset], train_mean, alpha=0.8, color='blue', linewidth=1, label='train')
ax.plot(df['timestamp'][num_trainset:], test_mean, alpha=0.8, color='green', linewidth=1, label='test')
ax.plot(df['timestamp'], df['value'], 'o', markersize=1, color='black', label='actual')
ax.fill_between(df['timestamp'], torch.cat([train_ci[0], test_ci[0]]), torch.cat([train_ci[1], test_ci[1]]), alpha=0.2, label='ci', color='gray')
ax.legend()


The training set has tight credible intervals, while that of the test set is wider. This makes sense since the credible interval gets larger if we extrapolate from the data. But still, we would like to see better performance in the forecasting part.


# Kernel Decomposition
We decompose additive kernels to see how each kernel functions.

In [ ]:
from sts.gp.graph import plot_components

x_all = torch.cat([df_train.tensor, x_test.tensor])
components = model.decompose_timeseries(x_all)
fig, ax = plot_components(df['timestamp'], components[1], y=model.predict(x_all), transform=transform)

# Evaluation
We measure mean absolute error (MAE), continuous-ranked probability score (CRPS), and log-likelihood (LL) using normalized values on both of the training set and the test set.

In [ ]:
train_mean, train_var, train_ci = get_mvn_stats(model.predict(df_train))
test_mean, test_var, test_ci = get_mvn_stats(model.predict(x_test))


Here we compute MAE for both training data and test data.

In [ ]:
from sts.metrics import MAE
MAE_train = MAE(y_train.tensor, train_mean).item()
MAE_test = MAE(y_test.tensor, test_mean).item()
print(f'Mean absolute error for training set is: {MAE_train:.3f}')
print(f'Mean absolute error for test set is: {MAE_test:.3f}')

Here we compute CRPS for both training data and test data.

In [ ]:
from sts.crps import crps_gaussian
crps_train = crps_gaussian(y_train.tensor, train_mean, train_var.sqrt())
crps_test = crps_gaussian(y_test.tensor, test_mean, test_var.sqrt())
print(f'Continuous ranked probability score for training set is: {crps_train.mean().item():.3f}')
print(f'Continuous ranked probability score for test set is: {crps_test.mean().item():.3f}')

Here we compute log-likelihood for both training data and test data.



In [ ]:
from sts.metrics import log_likelihood_error
ll_train = log_likelihood_error(train_mean, y_train.tensor, train_var).item()
ll_test = log_likelihood_error(test_mean, y_test.tensor, test_var).item()
print(f'Log-likelihood for training set is: {ll_train:.3f}')
print(f'Log-likelihood for test set is: {ll_test:.3f}')

In [ ]:
scores = [[MAE_train, crps_train.mean().item(), ll_train], [MAE_test, crps_test.mean().item(), ll_test]]
df_meature = pd.DataFrame(scores, columns = ['MAE', 'CPRS', 'LL'])
df_meature

Above is the table showing the values of each metrics, where the first line is for the training set and the second line is for the test set. As a kind reminder, MAE and CRPS are better if they are lower, while LL is better when it's higher.


At the same time, we noticed the training set shows higher accuracy than the test set. To improve this, we will assign priors to the hyperparameters and use the hierarchical priors to gain joint information of univarite time series.

# References

[1] Corani, G., Benavoli, A., Augusto, J. and Zaffalon, M., 2020. Automatic Forecasting using Gaussian Processes. arXiv preprint arXiv:2009.08102.